## Style Completion Task


In [2]:
%pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [29]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, Trainer, TrainingArguments, pipeline
import PyPDF2

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [7]:
def pdf_to_text(pdf_path, output_txt):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    with open(output_txt, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [8]:
pdf_path = '/content/content.pdf'
output_txt = 'content.txt'
pdf_to_text(pdf_path, output_txt)

In [12]:
pds_data = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/content.txt',
    block_size=32
)

In [13]:
pds_data[0], pds_data[0].shape

(tensor([47231,  6418,   286,  6060,  5800,   198, 20238,   262,  7605,   290,
         10688,   345,   761,   284,   923,   220,   198,  8601,  2565,   286,
           534,  1366,   198, 46200,   272, 18024,  9536,   343,   198,  3483,
         29138,  2751]),
 torch.Size([32]))

In [14]:
tokenizer.decode(pds_data[0])

'Principles of Data Science\nLearn the techniques and math you need to start \nmaking sense of your data\nSinan Ozdemir\nBIRMING'

In [15]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [19]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
collator_example = data_collator([tokenizer('I am an input'), tokenizer('So am I')])

collator_example

{'input_ids': tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50257]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]]), 'labels': tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])}

In [20]:
collator_example.input_ids

tensor([[   40,   716,   281,  5128],
        [ 2396,   716,   314, 50257]])

In [21]:
tokenizer.pad_token_id

50257

In [22]:
collator_example.attention_mask

tensor([[1, 1, 1, 1],
        [1, 1, 1, 0]])

In [26]:
tokenizer.pad_token_id

50257

In [27]:
collator_example.labels

tensor([[  40,  716,  281, 5128],
        [2396,  716,  314, -100]])

In [30]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

pretrained_generator = pipeline(
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_k': 10, 'top_p': 0.9, 'temperature': 0.7}
)


Device set to use cpu


In [31]:
print("=========")
for generated_sequence in pretrained_generator("A dataset shows the relationships", num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print("========")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A dataset shows the relationships between the time points between the two groups of variables. A set of data sets may be included as a basis for testing to determine whether the association is at rest. It cannot reveal any important aspects of the original dataset, which
A dataset shows the relationships between the five top political parties and all other major European Union states (with a total of 546,984 votes each), but there are several major differences. On the top one (European Union) most were in the
A dataset shows the relationships to the total age from age 1 through age 4.

The age ranges are given in meters

The values will be converted to the mean ages of those age ranges

The age ranges are given in hours



In [ ]:
training_args = TrainingArguments(
  output_dir='./gpt2-pds',
  overwrite_output_dir=True,
  num_train_epochs=3,
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  warmup_steps=len(pds_data.examples),
  gradient_accumulation_steps=16,
  evaluation_strategy='epoch',
  save_strategy='epoch',
  logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=pds_data.examples[:int(len(pds_data.examples)*.8)],
    eval_dataset=pds_data.examples[int(len(pds_data.examples)*.8):]
)

trainer.evaluate()

In [ ]:
trainer.train()